# 2D object detection task with FLaVor inference service

This guide will walk you through tailoring the FLaVor inference service for 2D object detection tasks using the model from [YOLOv8-Medical-Imaging](https://github.com/sevdaimany/YOLOv8-Medical-Imaging).

## Prerequisite

As for the working environment, please ensure you have the following dependencies installed:

```
python >= 3.10
ultralytics
numpy < 2.0.0
```

or simply run:


In [ ]:
!poetry install --with det_example

Next, download pretrain weight:

In [ ]:
# pwd: examples/inference
!wget https://github.com/sevdaimany/YOLOv8-Medical-Imaging/raw/master/runs/detect/train/weights/best.pt

## Implementation

### Setup imports

In [ ]:
import os
from typing import Any, Callable, Dict, List, Sequence, Tuple

import cv2
import numpy as np
from ultralytics import YOLO

from flavor.serve.apps import InferAPP
from flavor.serve.inference.data_models.api import (
    BaseAiCOCOImageInputDataModel,
    BaseAiCOCOImageOutputDataModel,
)
from flavor.serve.inference.data_models.functional import AiImage
from flavor.serve.inference.inference_models import BaseAiCOCOImageInferenceModel
from flavor.serve.inference.strategies import AiCOCODetectionOutputStrategy

### Setup inference model

In this section, we would create `ClassificationInferenceModel` inheriting from `BaseAiCOCOImageInferenceModel`. There are few abstract methods that we must override such as `define_inference_network`, `set_categories`, `set_regressions`, `data_reader` and `output_formatter`. For the inference process related methods such as `preprocess`, `inference` and `postprocess`, we override them if necessary. `preprocess` and `postprocess` would remain an identical operation if unmodified. `inference` by default runs `self.forward(x)`.

Firstly, we need to implement submethods: `define_inference_network`, `set_categories` and `set_regressions`. These are defined in the `__init__()` constructor of the parent class `BaseAiCOCOImageInferenceModel`. `define_inference_network` defines your inference network and loads its pre-trained weight. `set_categories` and `set_regressions` define category and regression information. For example, a classification output would contain `c` channels. We need to show the exact meaning of each channel by specifying in `set_categories`. Refer to the following example for more detail.

Next, we implement other submethods that would be used in the `__call__` function of our inference model. See below workflow.

### `__call__` function workflow for the inference model
![__call__](images/call.png "inference workflow")

In [ ]:
class DetectionInferenceModel(BaseAiCOCOImageInferenceModel):
    def __init__(self):
        self.formatter = AiCOCODetectionOutputStrategy()
        super().__init__()

    def define_inference_network(self) -> Callable:
        ckpt_path = os.path.join(os.getcwd(), "best.pt")
        if not os.path.exists(ckpt_path):
            from urllib.request import urlretrieve

            urlretrieve(
                "https://github.com/sevdaimany/YOLOv8-Medical-Imaging/raw/master/runs/detect/train/weights/best.pt",
                ckpt_path,
            )
        return YOLO(ckpt_path)

    def set_categories(self) -> List[Dict[str, Any]]:
        categories = [
            {"name": "RBC", "display": True},
            {"name": "WBC", "display": True},
            {"name": "Platelets", "display": True},
        ]
        return categories

    def set_regressions(self) -> None:
        return None

    def data_reader(self, files: Sequence[str], **kwargs) -> Tuple[np.ndarray, None, None]:
        image = cv2.imread(files[0])
        image = image.astype(np.float32)

        return image, None, None

    def inference(self, x: np.ndarray) -> Any:
        return self.network.predict(x, conf=0.7)[0]

    def postprocess(self, model_out: Any, **kwargs) -> Dict[str, Any]:

        format_output = {
            "bbox_pred": [],
            "cls_pred": [],
            "confidence_score": [],
        }

        for obj in model_out.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = obj
            format_output["bbox_pred"].append([int(x1), int(y1), int(x2), int(y2)])
            cls_pred = np.zeros(3)
            cls_pred[int(class_id)] = 1
            format_output["cls_pred"].append(cls_pred)
            format_output["confidence_score"].append(score)

        return format_output

    def output_formatter(
        self,
        model_out: Dict[str, Any],
        images: Sequence[AiImage],
        categories: Sequence[Dict[str, Any]],
        regressions: Sequence[Dict[str, Any]],
        **kwargs
    ) -> BaseAiCOCOImageOutputDataModel:
        output = self.formatter(
            model_out=model_out, images=images, categories=categories, regressions=regressions
        )
        return output

### Integration with InferAPP
We could integrate our defined inference model with FLaVor `InferAPP`, a FastAPI application. To initiate the application, users have to define `input_data_model` and `output_data_model` which are the standard input and output structure for the service. Then, provide `infer_function` as the main inference operation. After initiate the service, `/invocations` API end point would be available to process the inference request. We encourge users to implement a stand-alone python script based on this jupyter notebook tutorial.

#### (Optional) to initiate application in jupyter notebook, you have to run the following block.

In [ ]:
# This block is only for jupyter notebook. You don't need this in stand-alone script.
import nest_asyncio
nest_asyncio.apply()

#### Initiate the service

In [ ]:
app = InferAPP(
    infer_function=DetectionInferenceModel(),
    input_data_model=BaseAiCOCOImageInputDataModel,
    output_data_model=BaseAiCOCOImageOutputDataModel,
)

In [ ]:
app.run(port=int(os.getenv("PORT", 9111)))

### Send request
We can send request to the running server by `send_request.py` which opens the input files and the corresponding JSON file and would be sent via formdata. We expect to have response in AiCOCO format.

```bash
# pwd: examples/inference
python send_request.py -f test_data/det/BloodImage_00000_jpg.rf.5fb00ac1228969a39cee7cd6678ee704.jpg -d test_data/det/input.json
```

## Setup Dockerfile
In order to interact with other services, we have to wrap the inference model into a docker container. Here's an example of the dockerfile.

```dockerfile
FROM nvidia/cuda:12.2.2-runtime-ubuntu20.04

RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        python3 \
        python3-pip \
    && ln -sf /usr/bin/python3 /usr/bin/python
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends wget\

RUN pip install ultralytics
RUN pip install https://github.com/ailabstw/FLaVor/archive/refs/heads/release/stable.zip -U && pip install flavor

WORKDIR /app
COPY your_script.py  /app/

RUN wget https://github.com/sevdaimany/YOLOv8-Medical-Imaging/raw/master/runs/detect/train/weights/best.pt -P /app/

CMD ["python", "your_script.py"]

```